In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../src/')

In [4]:
pip install tables

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [5]:
APPLIANCE = ['fridge', 'dish_washer', 'washing_machine']
THRESHOLD = [50., 10., 20.]
MIN_ON = [1., 30., 30.]
MIN_OFF = [1., 3., 30.]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000

# UKDale training network

In [6]:
from utils.preprocessing import get_status
houses=[1,2,5]
ds_meter = []
ds_appliance = []
ds_status = []
for i in houses:
    ds = pd.read_feather('../data/ukdale/feather_files/UKDALE_%d_train.feather' %(i))
    ds.set_index('datetime', inplace=True)
  
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(3)]

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_train = [Power(ds_meter[i][:int(0.8*ds_len[i])], 
                        ds_appliance[i][:int(0.8*ds_len[i])], 
                        ds_status[i][:int(0.8*ds_len[i])], 
                        SEQ_LEN, BORDER, MAX_POWER, True) for i in range(3)]

ds_house_valid = [Power(ds_meter[i][int(0.8*ds_len[i]):], 
                        ds_appliance[i][int(0.8*ds_len[i]):],
                        ds_status[i][int(0.8*ds_len[i]):], 
                        SEQ_LEN, BORDER, MAX_POWER, False) for i in range(3)]

ds_house_total  = [Power(ds_meter[i], ds_appliance[i], ds_status[i], 
                         SEQ_LEN, BORDER, MAX_POWER, False) for i in range(3)]

ds_train = torch.utils.data.ConcatDataset([ds_house_train[0], ds_house_train[2]])
ds_valid = torch.utils.data.ConcatDataset([ds_house_valid[0], ds_house_valid[2]])

dl_train = DataLoader(dataset = ds_train, batch_size = BATCH_SIZE, shuffle=True, num_workers=2)
dl_valid = DataLoader(dataset = ds_valid, batch_size = BATCH_SIZE, shuffle=False, num_workers=2)
dl_test = DataLoader(dataset = ds_house_total[1], batch_size = BATCH_SIZE, shuffle=False, num_workers=2)

dl_house_test = [DataLoader(dataset = ds_house_total[1], batch_size = 1, shuffle=False)]
dl_house_valid = [DataLoader(dataset = ds_house_valid[i], batch_size = 1, shuffle=False) for i in [0,2]]
dl_house_total = [DataLoader(dataset = ds_house_total[i], batch_size = 1, shuffle=False) for i in range(3)]

In [ ]:
import torch.nn as nn
import torch.optim as optim
from utils.net import PTPNet
from utils.training import train_model

batch_size = BATCH_SIZE
n_epochs = 200

train_loader = dl_train
valid_loader = dl_valid
test_loader = dl_test
model = PTPNet(1,3,32).cuda()
optimizer = optim.Adam(model.parameters(), lr=5.E-5)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([1.0, 0.9, 0.9])).cuda()
fn='../data/ukdale/network_weights/UKDALE_network_weights.pth'
model, train_loss, valid_loss, test_loss = train_model(model, batch_size, n_epochs, fn, train_loader, valid_loader, test_loader, optimizer, criterion)


[  1/200] train_loss: 0.68969 valid_loss: 0.62037 test_loss: 0.61620 
Validation loss decreased (inf --> 0.620375).  Saving model ...
[  2/200] train_loss: 0.57958 valid_loss: 0.51104 test_loss: 0.49913 
Validation loss decreased (0.620375 --> 0.511041).  Saving model ...
[  3/200] train_loss: 0.48308 valid_loss: 0.43294 test_loss: 0.41793 
Validation loss decreased (0.511041 --> 0.432945).  Saving model ...
[  4/200] train_loss: 0.41068 valid_loss: 0.37957 test_loss: 0.35901 
Validation loss decreased (0.432945 --> 0.379569).  Saving model ...
[  5/200] train_loss: 0.36150 valid_loss: 0.34452 test_loss: 0.31808 
Validation loss decreased (0.379569 --> 0.344517).  Saving model ...
[  6/200] train_loss: 0.32845 valid_loss: 0.31971 test_loss: 0.29074 
Validation loss decreased (0.344517 --> 0.319713).  Saving model ...
[  7/200] train_loss: 0.30117 valid_loss: 0.29839 test_loss: 0.26757 
Validation loss decreased (0.319713 --> 0.298387).  Saving model ...
[  8/200] train_loss: 0.28015 va

# REFIT training network

## Fridge case

In [7]:
APPLIANCE = ['fridge']
THRESHOLD = [20.]
MIN_ON = [600. / 60]
MIN_OFF = [100. / 60]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [8]:
from utils.preprocessing import get_status
original_meter = []
ds_appliance = []
ds_status = []
houses=[2,5,9,12,15]
for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/fridge/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    original_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(original_meter[i]) for i in range(len(houses))]

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(original_meter[i], ds_appliance[i], ds_status[i],SEQ_LEN, BORDER, MAX_POWER, False) for i in range(len(houses))]
ds_house_train  = [Power(original_meter[i][:int(0.8*ds_len[i])], 
                         ds_appliance[i][:int(0.8*ds_len[i])], 
                         ds_status[i][:int(0.8*ds_len[i])],
                         SEQ_LEN, BORDER, MAX_POWER, True) for i in range(len(houses))]
ds_house_valid  = [Power(original_meter[i][int(0.8*ds_len[i]):], 
                         ds_appliance[i][int(0.8*ds_len[i]):], 
                         ds_status[i][int(0.8*ds_len[i]):],
                         SEQ_LEN, BORDER, MAX_POWER, False) for i in range(len(houses))]

ds_train      = torch.utils.data.ConcatDataset([ds_house_train[i] for i in range(len(houses)-1)])
ds_valid      = torch.utils.data.ConcatDataset([ds_house_valid[i] for i in range(len(houses)-1)])

dl_train = DataLoader(dataset = ds_train, batch_size = BATCH_SIZE, shuffle=False, num_workers=2)
dl_valid = DataLoader(dataset = ds_valid, batch_size = BATCH_SIZE, shuffle=False, num_workers=2)
dl_test = DataLoader(dataset = ds_house_total[-1], batch_size = BATCH_SIZE, shuffle=False, num_workers=2)

In [10]:
import torch.nn as nn
import torch.optim as optim
from utils.net import PTPNet
from utils.training import train_model

batch_size = BATCH_SIZE
n_epochs = 200

train_loader = dl_train
valid_loader = dl_valid
test_loader = dl_test

model = PTPNet(1,1,32).cuda()
optimizer = optim.Adam(model.parameters(), lr=5.E-5)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([1.5])).cuda()
fn = '../data/refit/network_weights/REFIT_fridge_network_weights.pth'
model, train_loss, valid_loss, test_loss = train_model(model, batch_size, n_epochs, fn, train_loader, valid_loader, test_loader, optimizer, criterion)

[  1/200] train_loss: 0.79067 valid_loss: 0.74895 test_loss: 0.71022 
Validation loss decreased (inf --> 0.748947).  Saving model ...
[  2/200] train_loss: 0.74336 valid_loss: 0.73062 test_loss: 0.66869 
Validation loss decreased (0.748947 --> 0.730620).  Saving model ...
[  3/200] train_loss: 0.70634 valid_loss: 0.66657 test_loss: 0.62075 
Validation loss decreased (0.730620 --> 0.666571).  Saving model ...
[  4/200] train_loss: 0.66884 valid_loss: 0.67297 test_loss: 0.61870 
[  5/200] train_loss: 0.64395 valid_loss: 0.64848 test_loss: 0.56347 
Validation loss decreased (0.666571 --> 0.648484).  Saving model ...
[  6/200] train_loss: 0.63165 valid_loss: 0.69651 test_loss: 0.61628 
[  7/200] train_loss: 0.62136 valid_loss: 0.67472 test_loss: 0.59341 
[  8/200] train_loss: 0.61342 valid_loss: 0.61578 test_loss: 0.51655 
Validation loss decreased (0.648484 --> 0.615778).  Saving model ...
[  9/200] train_loss: 0.59939 valid_loss: 0.63544 test_loss: 0.53530 
[ 10/200] train_loss: 0.60115 

## Dishwasher case

In [11]:
APPLIANCE = ['dishwasher']
THRESHOLD = [30.]
MIN_ON = [1400. / 60]
MIN_OFF = [1600. / 60]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [12]:
from utils.preprocessing import get_status
original_meter = []
ds_appliance = []
ds_status = []
houses=[1,3,5,6,7,9,10,11,13,15,16,18,20,2]

for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/dishwasher/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    original_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(original_meter[i]) for i in range(len(houses))]

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(original_meter[i], ds_appliance[i], ds_status[i],SEQ_LEN, BORDER, MAX_POWER, False) for i in range(len(houses))]
ds_house_train  = [Power(original_meter[i][:int(0.8*ds_len[i])], 
                         ds_appliance[i][:int(0.8*ds_len[i])], 
                         ds_status[i][:int(0.8*ds_len[i])],
                         SEQ_LEN, BORDER, MAX_POWER, True) for i in range(len(houses))]
ds_house_valid  = [Power(original_meter[i][int(0.8*ds_len[i]):], 
                         ds_appliance[i][int(0.8*ds_len[i]):], 
                         ds_status[i][int(0.8*ds_len[i]):],
                         SEQ_LEN, BORDER, MAX_POWER, False) for i in range(len(houses))]

ds_train      = torch.utils.data.ConcatDataset([ds_house_train[i] for i in range(len(houses)-1)])
ds_valid      = torch.utils.data.ConcatDataset([ds_house_valid[i] for i in range(len(houses)-1)])

dl_train = DataLoader(dataset = ds_train, batch_size = BATCH_SIZE, shuffle=False, num_workers=2)
dl_valid = DataLoader(dataset = ds_valid, batch_size = BATCH_SIZE, shuffle=False, num_workers=2)
dl_test = DataLoader(dataset = ds_house_total[-1], batch_size = BATCH_SIZE, shuffle=False, num_workers=2)

dl_house_total = [DataLoader(dataset = ds_house_total[i], batch_size = 1, shuffle=False) for i in range(len(houses))]

In [14]:
import torch.nn as nn
import torch.optim as optim
from utils.net import PTPNet
from utils.training import train_model

batch_size = BATCH_SIZE
n_epochs = 100

train_loader = dl_train
valid_loader = dl_valid
test_loader = dl_test

model = PTPNet(1,1,32).cuda()
optimizer = optim.Adam(model.parameters(), lr=5.E-5)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([1.9])).cuda()
fn = '../data/refit/network_weights/REFIT_dishwasher_network_weights.pth'
model, train_loss, valid_loss, test_loss = train_model(model, batch_size, n_epochs, fn, train_loader, valid_loader, test_loader, optimizer, criterion)

[  1/100] train_loss: 0.28475 valid_loss: 0.11968 test_loss: 0.20956 
Validation loss decreased (inf --> 0.119684).  Saving model ...
[  2/100] train_loss: 0.14600 valid_loss: 0.09399 test_loss: 0.17118 
Validation loss decreased (0.119684 --> 0.093994).  Saving model ...
[  3/100] train_loss: 0.13236 valid_loss: 0.08466 test_loss: 0.14345 
Validation loss decreased (0.093994 --> 0.084663).  Saving model ...
[  4/100] train_loss: 0.12355 valid_loss: 0.08342 test_loss: 0.13584 
Validation loss decreased (0.084663 --> 0.083422).  Saving model ...
[  5/100] train_loss: 0.12522 valid_loss: 0.08108 test_loss: 0.13313 
Validation loss decreased (0.083422 --> 0.081081).  Saving model ...
[  6/100] train_loss: 0.12170 valid_loss: 0.08034 test_loss: 0.12393 
Validation loss decreased (0.081081 --> 0.080339).  Saving model ...
[  7/100] train_loss: 0.11863 valid_loss: 0.07584 test_loss: 0.11985 
Validation loss decreased (0.080339 --> 0.075841).  Saving model ...
[  8/100] train_loss: 0.11050 va

## Washing-machine case

In [34]:
APPLIANCE = ['washing_machine']
THRESHOLD = [20.]
MIN_ON = [1600. / 60]
MIN_OFF = [800. / 60]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000.

In [35]:
from utils.preprocessing import get_status
original_meter = []
ds_appliance = []
ds_status = []
houses = [1,2,3,5,7,8,9,10,11,13,15,16,17,18,19,20,21,6]


for i in houses:
    ds = pd.read_feather('../data/refit/feather_files/washing_machine/REFIT_%d.feather' %i)
    ds.set_index('datetime', inplace=True)
    
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    original_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(original_meter[i]) for i in range(len(houses))]

In [36]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(original_meter[i], ds_appliance[i], ds_status[i],SEQ_LEN, BORDER, MAX_POWER, False) for i in range(len(houses))]
ds_house_train  = [Power(original_meter[i][:int(0.8*ds_len[i])], 
                         ds_appliance[i][:int(0.8*ds_len[i])], 
                         ds_status[i][:int(0.8*ds_len[i])],
                         SEQ_LEN, BORDER, MAX_POWER, True) for i in range(len(houses))]
ds_house_valid  = [Power(original_meter[i][int(0.8*ds_len[i]):], 
                         ds_appliance[i][int(0.8*ds_len[i]):], 
                         ds_status[i][int(0.8*ds_len[i]):],
                         SEQ_LEN, BORDER, MAX_POWER, False) for i in range(len(houses))]

ds_train      = torch.utils.data.ConcatDataset([ds_house_train[i] for i in range(len(houses)-1)])
ds_valid      = torch.utils.data.ConcatDataset([ds_house_valid[i] for i in range(len(houses)-1)])

dl_train = DataLoader(dataset = ds_train, batch_size = BATCH_SIZE, shuffle=False, num_workers=2)
dl_valid = DataLoader(dataset = ds_valid, batch_size = BATCH_SIZE, shuffle=False, num_workers=2)
dl_test = DataLoader(dataset = ds_house_total[-1], batch_size = BATCH_SIZE, shuffle=False, num_workers=2)

In [37]:
import torch.nn as nn
import torch.optim as optim
from utils.net import PTPNet
from utils.training import train_model

batch_size = BATCH_SIZE
n_epochs = 200

train_loader = dl_train
valid_loader = dl_valid
test_loader = dl_test

model = PTPNet(1,1,32).cuda()
optimizer = optim.Adam(model.parameters(), lr=5.E-5)
#criterion = nn.BCEWithLogitsLoss().cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.FloatTensor([2.9])).cuda()
fn = '../data/refit/network_weights/REFIT_washingmachine_network_weights.pth'
model, train_loss, valid_loss, test_loss = train_model(model, batch_size, n_epochs, fn, train_loader, valid_loader, test_loader, optimizer, criterion)

[  1/200] train_loss: 0.31964 valid_loss: 0.22513 test_loss: 0.11162 
Validation loss decreased (inf --> 0.225135).  Saving model ...
[  2/200] train_loss: 0.20309 valid_loss: 0.19694 test_loss: 0.08264 
Validation loss decreased (0.225135 --> 0.196940).  Saving model ...
[  3/200] train_loss: 0.18170 valid_loss: 0.17775 test_loss: 0.07107 
Validation loss decreased (0.196940 --> 0.177750).  Saving model ...
[  4/200] train_loss: 0.16533 valid_loss: 0.16160 test_loss: 0.06801 
Validation loss decreased (0.177750 --> 0.161597).  Saving model ...
[  5/200] train_loss: 0.16097 valid_loss: 0.15898 test_loss: 0.06337 
Validation loss decreased (0.161597 --> 0.158979).  Saving model ...
[  6/200] train_loss: 0.16022 valid_loss: 0.16080 test_loss: 0.06329 
[  7/200] train_loss: 0.14983 valid_loss: 0.15092 test_loss: 0.05631 
Validation loss decreased (0.158979 --> 0.150919).  Saving model ...
[  8/200] train_loss: 0.15451 valid_loss: 0.14392 test_loss: 0.05520 
Validation loss decreased (0.15